# EPH - Individuos


## Ajuste de ruta del proyecto en Notebook

In [1]:
import sys

# Ajusto para que la reconozca la ruta del proyecto
sys.path.append("..") 

## Unificación de los archivos de Individuos

In [2]:
from src.utils.constants import DATA_SOURCE_DIR,DATA_CLEAN_DIR,FILENAME_INDIVIDUOS_UNIFIED
from src.utils.helpers import  process_file, save_to_txt

# Procesar y guardar individuos
encabezados,individual = process_file(DATA_SOURCE_DIR, category="individual")
save_to_txt(encabezados,individual, DATA_CLEAN_DIR , FILENAME_INDIVIDUOS_UNIFIED)


✅ Archivo TXT guardado en: C:\Users\galle\OneDrive\Escritorio\Python-Estudio\code\data\clean\individuos_unificado.txt


## Procesamiento de los archivos

Abro el archivo unificado

In [3]:
from src.utils.constants import INDIVIDUOS_UNIFIED_DIR
from src.utils.helpers import read_file_dic

header,data=read_file_dic(INDIVIDUOS_UNIFIED_DIR)

Proceso el archivo

In [4]:
import src.procesamientos.individuos as indiv

indiv.add_extra_data(header,data)

Guardo los datos procesados a un archivo

In [5]:
from src.utils.constants import INDIVIDUOS_PROCESSED_DIR
from src.utils.helpers import save_to_csv

save_to_csv(INDIVIDUOS_PROCESSED_DIR,header,data, delimiter=";")

## Análisis de Individuos

Abro el archivo de individuos procesado:

In [6]:
from src.utils.helpers import read_file_dic

header,data=read_file_dic(INDIVIDUOS_PROCESSED_DIR)

Importo las funciones para realizar las consultas

In [15]:
import src.consultas.consultas as cs

In [18]:
import importlib

importlib.reload(cs)

<module 'src.consultas.consultas' from 'c:\\Users\\galle\\OneDrive\\Escritorio\\Python-Estudio\\code\\notebooks\\..\\src\\consultas\\consultas.py'>

### 1- A partir de la información de cada año contenida en el dataset se debe informar, año tras año, el porcentaje de personas mayores a 6 años capaces e incapaces de leer y escribir.

In [22]:
cs.cant_personas_alfabetizadas(data)

Año           % Alfabetos      % No Alfabetos
---------------------------------------------
2024                98.28                1.72
2023                98.33                1.67


### 2- A partir de un año y trimestre elegido por el usuario informar el porcentaje de personas no nacidas en Argentina que hayan cursado un nivel universitario o superior

In [ ]:
#Ingreso de datos por el usuario
ano=input("Ingrese el año: ")
trim=input("Ingrese el trimestre [1 a 4]: ")

cs.porc_extranjero_universitario(ano,trim,data)

### 3- A partir de la información contenida en el dataset informar el año y trimestre donde hubo menor desocupación.

In [ ]:
cs.info_menor_desocupacion(data)

### 4- Ranking de los 5 aglomerados con mayor porcentaje de hogares con dos o más ocupantes con estudios universitarios o superiores finalizados. Información obtenida a partir del par de archivos más recientes. 

In [ ]:
# 1. Cuento cuántos universitarios hay por hogar y guardar pondera

universitarios_por_hogar, pondera_por_hogar = cs.contar_universitarios_y_pondera_por_hogar(data)

# 2. Filtro hogares con al menos 2 universitarios

hogares_filtrados = cs.filtrar_hogares_con_min_universitarios(universitarios_por_hogar,pondera_por_hogar,min_universitarios=2)

In [ ]:
# 3. Suma ponderada de todos los hogares

total_hogares = cs.contar_hogares(pondera_por_hogar)

# 4. Suma poderada de hogares con 2+ universitarios por aglomerado

hogares_con_universitarios_min = cs.contar_hogares(hogares_filtrados)

In [ ]:
# 5. Calculo porcentajes  y ordeno por porcentajes
# También se imprime el nombre del aglomerado, el total de hogares y el total de hogares con al 
# menos 2 universitarios

top_5_aglomerados =cs.obtener_top_n_porcentaje_hogares_universitarios(total_hogares, hogares_con_universitarios_min )

In [ ]:
# 7. Impresión del ranking de 5 aglomerados con mayor porcentaje de hogares con 2 o más universitarios
cs.imprimir_ranking_aglomerados(top_5_aglomerados, cantidad=5)

### 7- Informar para cada aglomerado el porcentaje de personas mayores que hayan cursado al menos en nivel universitario o superior.

Consideramos como muestra total para el cálculo porcentual sólo a las personas mayores de edad, que son las que tienen la posibilidad real de poder alcanzarlo.


In [ ]:
cs.info_porcentual_educacionsuperior_aglomerado(data)

### 9- Pedir al usuario que seleccione un aglomerado y a partir de la información contenida retornar una tabla que contenga la cantidad de personas mayores de edad según su nivel de estudios alcanzados.


In [19]:
aglomerado = input("Ingrese el aglomerado: ")

cs.tabla_nivel_educativo(data, aglomerado)

Aglomerado     32 - Ciudad Autónoma de Buenos Aires
**************************************************************************************************************************************************************************************************************************************************************************************************************************************************************
Año     Trimestre   Primario incompleto / Ed. especial      Primario completo                       Secundario incompleto                   Secundario completo                     Superior universitario incompleto       Superior universitario completo         Sin instrucción                         
********************************************************************************************************************************************************************************************************************************************************************************************************

### 10- Pedir al usuario que seleccione dos aglomerados y a partir de la información contenida retornar una tabla que contenga el porcentaje de personas mayores de edad con secundario incompleto.

In [10]:

aglo_input1 = int(input("Ingrese un aglomerado: "))
aglo_input2 = int(input("Ingrese otro aglomerado: "))

tabla_porcentaje = cs.personas_secundario_incompleto_anio_trimestre(aglo_input1, aglo_input2, data)

cs.imprimir_porcentaje_secundario_incompleto(tabla_porcentaje)

  Año    Trimestre     Aglomerado A         Aglomerado B    
------------------------------------------------------------
  2023      3            8.61 %              17.09 %       
  2024      4            7.23 %              17.03 %       
